<h1>[공공]서울시 따릉이 언제 얼만큼 탈까 베이스라인<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#데이터-다운로드" data-toc-modified-id="데이터-다운로드-1">데이터 다운로드</a></span></li><li><span><a href="#데이터-부르기" data-toc-modified-id="데이터-부르기-2">데이터 부르기</a></span></li><li><span><a href="#선형회귀-모델로-예측하기" data-toc-modified-id="선형회귀-모델로-예측하기-3">선형회귀 모델로 예측하기</a></span></li><li><span><a href="#랜덤포레스트-모델로-예측하기" data-toc-modified-id="랜덤포레스트-모델로-예측하기-4">랜덤포레스트 모델로 예측하기</a></span></li><li><span><a href="#submission-만들기" data-toc-modified-id="submission-만들기-5">submission 만들기</a></span></li></ul></div>

# 데이터 다운로드

DACON의 '[공공]서울시 따릉이 언제 얼만큼 탈까' 데이터로 베이스라인을 만들었다.

데이터 파일(train.csv / test.csv / submission.csv)은 데이콘에 있다.

https://dacon.io/competitions/open/235576/overview/

# 데이터 부르기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 한글 출력 설정
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

In [2]:
# 데이터를 불러오자

train = pd.read_csv('train.csv')
train.fillna(train.mean(), inplace=True)
train.head()

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
0,3,20,16.3,1.0,1.5,89.0,576.0,0.027,76.0,33.0,49.0
1,6,13,20.1,0.0,1.4,48.0,916.0,0.042,73.0,40.0,159.0
2,7,6,13.9,0.0,0.7,79.0,1382.0,0.033,32.0,19.0,26.0
3,8,23,8.1,0.0,2.7,54.0,946.0,0.040,75.0,64.0,57.0
4,9,18,29.5,0.0,4.8,7.0,2000.0,0.057,27.0,11.0,431.0


In [3]:
test = pd.read_csv('test.csv')
test.fillna(test.mean(), inplace=True)
test.head()

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5
0,0,7,20.7,0.0,1.3,62.0,954.0,0.041,44.0,27.0
1,1,17,30.0,0.0,5.4,33.0,1590.0,0.061,49.0,36.0
2,2,13,19.0,1.0,2.1,95.0,193.0,0.020,36.0,28.0
3,4,6,22.5,0.0,2.5,60.0,1185.0,0.027,52.0,38.0
4,5,22,14.6,1.0,3.4,93.0,218.0,0.041,18.0,15.0


In [4]:
print(train.shape)
print(test.shape)

(1459, 11)
(715, 10)


In [5]:
# train_df가 column수가 하나 더 많다. label 인지 확인해보자
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 11 columns):
id                        1459 non-null int64
hour                      1459 non-null int64
hour_bef_temperature      1459 non-null float64
hour_bef_precipitation    1459 non-null float64
hour_bef_windspeed        1459 non-null float64
hour_bef_humidity         1459 non-null float64
hour_bef_visibility       1459 non-null float64
hour_bef_ozone            1459 non-null float64
hour_bef_pm10             1459 non-null float64
hour_bef_pm2.5            1459 non-null float64
count                     1459 non-null float64
dtypes: float64(9), int64(2)
memory usage: 125.5 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 715 entries, 0 to 714
Data columns (total 10 columns):
id                        715 non-null int64
hour                      715 non-null int64
hour_bef_temperature      715 non-null float64
hour_bef_precipitation    715 non-null float64
hour_bef_windspeed     

train 의 count 열을 label로 두고 모델을 학습시킨다. 

학습시킨 모델에 validation 을 넣어 예측하며 모델 간 성능을 비교한다. 

가장 성능이 좋은 모델에 test를 넣어 예측한다.

In [6]:
# train 의 count 열을 label로 두자
x = train.drop(['count'], axis=1)
y = train['count']

# train 를 train 부분과 validation 부분으로 나누자
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x,y,test_size=0.2, random_state=42)

In [7]:
# train 데이터가 8:2로 나뉜 것을 알 수 있다.
print('나뉘기 전 : ', x.shape)
print('나뉜 후 train : ', x_train.shape)
print('나뉜 후 validation : ', x_val.shape)

나뉘기 전 :  (1459, 10)
나뉜 후 train :  (1167, 10)
나뉜 후 validation :  (292, 10)


# 선형회귀 모델로 예측하기

In [8]:
from sklearn.linear_model import LinearRegression

model_reg = LinearRegression()
model_reg.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [9]:
y_val_pred = model_reg.predict(x_val)

In [10]:
# MSE로 평가해보자
from sklearn.metrics import mean_squared_error

MSE_reg = mean_squared_error(y_val, y_val_pred)
print(f'MSE_reg : {MSE_reg}')

MSE_reg : 2736.965479262694


# 랜덤포레스트 모델로 예측하기

In [11]:
from sklearn.ensemble import RandomForestRegressor

model_forest = RandomForestRegressor(random_state=42)
model_forest.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=42,
           verbose=0, warm_start=False)

In [12]:
y_val_pred = model_forest.predict(x_val)

In [13]:
# MSE로 평가해보자
MSE_forest = mean_squared_error(y_val, y_val_pred)
print(f'MSE_forest : {MSE_forest}')

MSE_forest : 1857.719828767123


In [14]:
# 어떤 모델이 더 좋을까?
print(f'MSE_reg : {MSE_reg}')
print(f'MSE_forest : {MSE_forest}')

MSE_reg : 2736.965479262694
MSE_forest : 1857.719828767123


랜덤포레스트회귀가 더 좋아 보인다. test 데이터에 넣어서 제출하자

In [15]:
pred = model_forest.predict(test)

# submission 만들기

In [16]:
submission = pd.DataFrame(data=pred, columns=['count'])
submission['id'] = test['id']
submission=submission.set_index('id')
submission.to_csv('my_submission.csv', index=True)